In [ ]:
import pandas as pd
import numpy as np
from fitparse import FitFile

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

In [ ]:
## TODO: add interactive widgets to set these default values

MAX_HR = 180
DEFAULT_ZONES = {0:'lightgrey', 50:'silver', 60:'mediumturquoise', 70:'yellowgreen', 80:'gold', 90:'crimson'}

def zone_bins(zones = DEFAULT_ZONES, max_hr = MAX_HR):
    bin_values = [max_hr*z/100 for z in zones.keys()] + [np.inf]
    bin_names = ["{}%-{}%".format(a,b) if b != np.inf else "{}%+".format(a) for (a,b) in list(zip(list(zones.keys()), list(zones.keys())[1:]+[np.inf]))]
    return (bin_values,bin_names)

In [ ]:
def get_session_time(file_path):
    """A function to return the `time_created` field from a FIT file
    
    Parameters
    ----------
    file_path : str
        The file location of the FIT file

    Returns
    -------
    datetime
        <class 'datetime.datetime'> representation of when the FIT file was created on
    """
    with FitFile(open(file_path, 'rb')) as fitfile:
        fitfile._parse_message() # parse first message as header
        return fitfile._parse_message().get_value('time_created') # parse second message and look for `time_created`
    return None

In [ ]:
def get_sessions():
    """A function to return a list of FIT sessions in the data directory with its creation time.

    Returns
    -------
    list
        A list of tuples with the file path as first element and its creation time as the second element.
    """
    datafiles = !ls data/*.fit
    sessions = []
    for filepath in datafiles:
        sessions.append((filepath,get_session_time(filepath)))
    return sessions

In [ ]:
USE_COLUMNS = ["timestamp","distance","enhanced_speed","heart_rate","cadence","position_lat","position_long","enhanced_altitude"]
RENAME_COLUMNS = {"enhanced_speed": "speed", "enhanced_altitude": "altitude"}

def load_session_data(file_path, max_hr, zones):
    """A function to return the `time_created` field from a FIT file
    
    Some columns are dropped and renamed. Only keep rows where heart_rate data exists.
    Resample to 1 second frequency with interpolated values.
    
    Parameters
    ----------
    file_path : str
        The file location of the FIT file
    max_hr : int
        Maximum heart rate used for analysis
    zones : list
        List of heart rate zones definition

    Returns
    -------
    dataframe
        Pandas DataFrame of the FIT file
    """
    with FitFile(open(file_path, 'rb')) as fitfile:
        df = pd.DataFrame([record.get_values() for record in fitfile.get_messages('record')])
        df = df.loc[:,df.columns.isin(USE_COLUMNS)]
        for c in USE_COLUMNS:
            if c not in df.columns:
                df[c] = np.nan 
        df.rename(columns=RENAME_COLUMNS, inplace=True)
        df = df.dropna(subset=['heart_rate'])
        df.set_index('timestamp', inplace=True)
        df = df.resample('s').interpolate()
        df.heart_rate = df.heart_rate.rolling('20s').mean()
        bin_values, bin_names = zone_bins(zones, max_hr)
        df['hr_binned'] = pd.cut(df.heart_rate, bin_values, labels=bin_names)
        df['hr_zone_color'] = pd.cut(df.heart_rate, bin_values, labels=zones.values())
        return df    

In [ ]:
def figure_for_heart_rate_timeseries(df):
    """A function to return figure for the heart rate time series
        
    Parameters
    ----------
    df
        The Pandas DataFrame used.  Columns required: timestamp as index, heart_rate, hr_zone_color

    Returns
    -------
    figure
        Plotly figure
    """
    fig = go.Figure()
    for color, hrzc_df in df.groupby('hr_zone_color'):
        if len(hrzc_df) == 0:
            continue
        fig.add_trace(go.Bar(x=hrzc_df.index,
                           y=hrzc_df.heart_rate,
                           name=hrzc_df.iloc[0].hr_binned,
                           marker={'color': hrzc_df.hr_zone_color, 'line.width':0}))

    fig.add_trace(go.Scatter(x=df.index, y=df.heart_rate,
                             line=dict(color='darkgrey', width=1), showlegend=False))

    fig.update_layout(
        title = 'Heart Rate Zones with 20s Rolling Mean',
        xaxis = dict(title='Date and Time'),
        yaxis = dict(title='Heart Rate (bpm)'),
        bargap=0
    )
    return fig

In [ ]:
def figure_for_heart_rate_distribution(df):
    """A function to return figure for the heart rate zone distribution
        
    Parameters
    ----------
    df
        The Pandas DataFrame used.  Columns required: timestamp as index, heart_rate, hr_zone_color

    Returns
    -------
    figure
        Plotly figure
    """
    df = df.hr_binned.value_counts(sort=False)/60
    fig = go.Figure(go.Bar(
                x=df, y=df.index, orientation='h',
                marker_color=list(DEFAULT_ZONES.values())
    ))

    fig.update_layout(
        title = 'Heart Rate Zones Distribution',
        xaxis = dict(title='Time in Zone (minutes)'),
        yaxis = dict(title='Heart Rate Zone'),
    )
    return fig

In [ ]:
sessions = get_sessions()
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Heart Rate Dashboard"),
    html.Label([
        "Workout Sessions",
        dcc.Dropdown(
            id='sessions_dropdown', clearable=False,
            value = sessions[0][0],
            options=[
                {'label': d, 'value': f}
                for f,d in sessions
            ])
    ]),
    html.Div(id='max_hr_display', children="Maximum Heart Rate: 180"),
    dcc.Slider(
        id='max_heart_rate',
        min=130,
        max=210,
        step=1,
        value=MAX_HR,
        marks={x:'{} bpm'.format(x) if x % 10 == 0 else '' for x in range(130,211,5)},
    ),
    html.Button("Update Settings",
                id='save_config'),
    html.Div(id='output_text', children=""),
    dcc.Graph(
        id='heart_rate_timeseries',
        figure=go.Figure()
    ),
    dcc.Graph(
        id='heart_rate_distribution',
        figure=go.Figure()
    ),
])


@app.callback(
    Output('max_hr_display', 'children'),
    Input("max_heart_rate", "value"),
)
def changed_hr(max_hr):
    return "Maximum Heart Rate: {}".format(max_hr)


@app.callback(
    Output('output_text', 'children'),
    Output('heart_rate_timeseries', 'figure'),
    Output('heart_rate_distribution', 'figure'),
    Input("sessions_dropdown", "value"),
    Input("save_config", "n_clicks"),
    state=[State('max_heart_rate', 'value'),],
)
def change_session(session, n_clicks, max_hr):
    df = load_session_data(session, max_hr, DEFAULT_ZONES)
    start = df.index.min()
    end = df.index.max()
    output_text__children = "Session from {} to {}, duration: {}. Max heart rate: {}".format(start,end,end-start, max_hr)
    heart_rate_timeseries__figure = figure_for_heart_rate_timeseries(df)
    
    return (output_text__children,
            figure_for_heart_rate_timeseries(df),
            figure_for_heart_rate_distribution(df),
           )

# Run app and display result inline in the notebook
app.run_server(mode='external')

In [ ]:
for s in get_sessions():
#     if s[0] == "data/3042255479.fit":
    df = load_session_data(s[0], MAX_HR, DEFAULT_ZONES)
    display(df.head())
    break